<img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width="80%" />

<hr style="height:2px;border:none"/>
<h1 align='center'> INF-398 Introducción al Aprendizaje Automático I-2024 </h1>

<H3 align='center'> Taller 3 </H3>

<H5 align='center'>  </H3>

<hr style="height:2px;border:none"/>


## Integrantes: 


* Natalia Barraza 
* Cristian Marín

# Temas

* SVM
* Árboles
* KNN
* Ensembles


# Reglas & Formalidades

* Equipos de 2 integrantes.
* Pueden reusar código visto en clases y/o recolectar código/ideas de otros sitios, mencionando al autor y entregando un link a la fuente.
* Si resulta necesaria, la intervención de personas ajenas al grupo (e.g. experto) debe ser declarada y justificada.
* Tener roles dentro del equipo está bien, pero al final del proceso, cada miembro debe entender y estar en condiciones de exponer todo el trabajo realizado.

# Entregables




> * **Video:** Se debe preparar un video explicativo de **máx 15 minutos** donde se describe la metodología utilizada, los resultados obtenidos y las conclusiones de la experiencia.

> * **Código:** Se debe enviar un jupyter notebook con el código utilizado, de modo que sea posible **reproducir los resultados** presentados. Como alternativa, se puede entregar un link Github con el código fuente, incluyendo instrucciones precisas para ejecutar los experimentos. En cualquier caso (notebook o repo) el código debe estar ordenado y seccionado apropiadamente.

> * **Conformidad Ética:** Se debe incluir una breve declaración ética en que se indique que el trabajo que se está enviando es un trabajo original, desarollado por los autores en conformidad con todas reglas antes mencionadas. Se debe también mencionar brevemente cuál fue la contribución de cada miembro del equipo. La declaración puede ser parte del notebook o estar en un archivo dentro del repo.

> * **Defensa en vivo:** El día de clases agendado para la discusión del taller, se seleccionarán aleatoriamente algunos equipos que presentarán oralmente su trabajo ante el curso. Los autores serán evaluados considerando la discusión y debate que generen entre sus pares. Los puntos obtenidos (positivos o negativos) se sumarán a la nota final de taller.

# Fechas

> * Defensas: 11 de julio, horario de clases.
> * Fecha de entrega de vídeo y Jupyter (notebook): 11 de julio 23:59 Hrs.



# Instrucciones

EL taller consiste en:

>  **Desafío Kaggle (100%)**. Para esta parte, los autores enfrentarán un desafío en la plataforma Kaggle y serán calificados en base a su posición en el tablero de resultados y el puntaje obtenido.


# Desafío Kaggle



Para esta parte del taller, tendrán que constuir un modelo capaz de predecir el precio de una vivienda, considerando los atributos entregados, como cantidad de piezas, ubicacion, etc.
Algunas consideraciones:


*   Los unicos modelos que **no** estarán permitidos son las **Redes Neuronales y lo relacionado a Deep Learning.**
*   Se debe efectuar comparacion de modelos, ya sea un solo modelo y el efecto de la modificacion de hiperparametros o diferentes modelos.
* Deben incluir una Tabla con resultados finales, resultado de la comparacion.
* **Deben considerar como alternativa lo visto en T-12 Ensamblados**.




Para acceder a loss detalles del desafío ingrese al siguiente link:

https://www.kaggle.com/t/027c57308813443182b66ab44e14f99b

**Evaluacion**

Según las posiciones en el tablero privado de Kaggle, se asignarán los siguientes puntos adicionales:

* 1er Lugar = +5 pts.
* 2do y 3er Lugar = +3 pts.
* 4to y 5to Lugar = +2 pts.
* 6to y 7mo Lugar = +1 pts.
* 8vo o más = +0 pts.



* No hacer submission a Kaggle = -10 pts.
* No superar el puntaje baseline = -5 pts.


# Librerias

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import joblib

## Recursos para procesar texto

In [ ]:
# Descargar recursos necesarios para NLTK (ejecutar una vez) 
nltk.download('punkt')
nltk.download('stopwords')

# Preprocesamiento de texto

In [ ]:
df_train = pd.read_csv('text-train.csv')
df_test = pd.read_csv('text-test.csv')

# Inicializar el stemmer y la lista de stopwords para italiano
stemmer = SnowballStemmer('italian')         #reduce las palabras
stop_words = set(stopwords.words('italian')) #elimina conectores en italianno

def preprocess_text_italian(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z\sàèéìíîòóùú\s]', '', text, re.I|re.A)                  #eliminar todo lo que no sea letras
        text = text.lower().strip()                                                     #dejamos todo en minúscula y elimina espacios en blanco al inicio y al final
        tokens = nltk.word_tokenize(text, language='italian')                           #Separa el string y lo convierte en un string por cada palabra 
        tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]   #reduce la palabra solo si no es una palabra 'vacía'
        preprocessed_text = ' '.join(tokens)                                            #Se devuelve todo en un solo string
        return preprocessed_text
    else:
        return ''

# Aplicar preprocesamiento a las columnas 'title' y 'description' de df_train y df_test
df_train['title_clean'] = df_train['title'].apply(preprocess_text_italian)
df_train['description_clean'] = df_train['description'].apply(preprocess_text_italian)
df_test['title_clean'] = df_test['title'].apply(preprocess_text_italian)
df_test['description_clean'] = df_test['description'].apply(preprocess_text_italian)

#Se junta el título y la descripción preprocesada
X_train = df_train['title_clean'] + ' ' + df_train['description_clean']
y_train = df_train['class']
X_test = df_test['title_clean'] + ' ' + df_test['description_clean']

# Entrenamiento y comparación de los modelos

In [ ]:
# Listas de modelos y parámetros a probar
models = [
    ('SVM', SVC(kernel='linear', C=1, random_state=42), 'linear'),
    ('SVM', SVC(kernel='rbf', C=1, random_state=42), 'rbf'),
    ('SVM', SVC(kernel='poly', C=1, random_state=42), 'poly'),
    ('KNN', KNeighborsClassifier(n_neighbors=3), 3),
    ('KNN', KNeighborsClassifier(n_neighbors=5), 5),
    ('KNN', KNeighborsClassifier(n_neighbors=7), 7),
    ('RandomForest', RandomForestClassifier(n_estimators=50, random_state=42), 50),
    ('RandomForest', RandomForestClassifier(n_estimators=100, random_state=42), 100),
    ('RandomForest', RandomForestClassifier(n_estimators=200, random_state=42), 200)
]

# Tabla comparativa de resultados
results = []

for model_name, model, param in models:
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        (model_name.lower(), model)
    ])
    
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    f1_scores = cross_val_score(pipeline, X_train_split, y_train_split, cv=3, scoring='f1_macro')
    mean_f1 = f1_scores.mean()
    
    results.append((model_name, param, mean_f1))
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    results_df = pd.DataFrame({'id': df_test.index, 'class': y_pred})
    results_df.to_csv(f'resultados_clasificacion_{model_name.lower()}_{param}.csv', index=False)
    
    joblib.dump(pipeline, f'{model_name.lower()}_{param}_model.pkl')

# Crear DataFrame con resultados comparativos
results_df = pd.DataFrame(results, columns=['Model', 'Parameter', 'F1 Score'])
results_df.to_csv('comparativa_resultados.csv', index=False)

print(results_df)

In [4]:
results_df = pd.read_csv('MODELOS FINALES/comparativa_resultados.csv')  
results_df  

,Model,Parameter,F1 Score
0,SVM,linear,0.909054
1,SVM,rbf,0.903913
2,SVM,poly,0.801330
3,KNN,3,0.833350
4,KNN,5,0.828724
5,KNN,7,0.821660
6,RandomForest,50,0.903600
7,RandomForest,100,0.906338
8,RandomForest,200,0.905288


## Uso del ensamble con 3 distintos modelos 

In [ ]:
# Dividir X_train e y_train en conjuntos de entrenamiento y validación
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Definir los modelos que quieres incluir en el VotingClassifier
svm_linear = Pipeline([
    ('tfidf_svm', TfidfVectorizer()),
    ('svm', SVC(kernel='linear', C=1, random_state=42))
])

knn_3 = Pipeline([
    ('tfidf_knn', TfidfVectorizer()),
    ('knn', KNeighborsClassifier(n_neighbors=3))
])

random_forest_100 = Pipeline([
    ('tfidf_rf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Crear el VotingClassifier
voting_clf = VotingClassifier(estimators=[
    ('svm_linear', svm_linear),
    ('knn_3', knn_3),
    ('random_forest_100', random_forest_100)
], voting='hard')

# Entrenar el VotingClassifier con el conjunto de entrenamiento
voting_clf.fit(X_train_split, y_train_split)

# Predecir con el conjunto de validación
y_val_pred = voting_clf.predict(X_val)

# Calcular la métrica F1 en el conjunto de validación
f1_val = f1_score(y_val, y_val_pred, average='weighted')
print(f"Métrica F1 en el conjunto de validación: {f1_val}")

# Entrenar el VotingClassifier con todos los datos de entrenamiento
voting_clf.fit(X_train, y_train)

# Predecir con el VotingClassifier entrenado en el conjunto de prueba final
y_pred_voting = voting_clf.predict(X_test)

# Guardar los resultados de la votación mayoritaria
results_voting_df = pd.DataFrame({'id': df_test.index, 'class': y_pred_voting})
results_voting_df.to_csv('resultados_clasificacion_voting.csv', index=False)

# Guardar el modelo de votación mayoritaria
joblib.dump(voting_clf, 'voting_model.pkl')

print("Resultados de la clasificación por mayoría de votos guardados en 'resultados_clasificacion_voting.csv'")
print("Modelo de votación por mayoría guardado en 'voting_model.pkl'")


El resultado de la métrica f1 en el voting fue de 0.9413215403982274 en el conjunto de validación (es lo que entrega el print aprox. si se llegase a ejecutar el código)